##  Орищенко Виталий
## @Vitosh992

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool

In [2]:
df_col = { # Создадим названия для столбцов (оценки по дисциплинам)
    'MA' ,
    'Phys' ,
    'Math_modeling' ,
    'Web_programming' ,
    'db_engineering' ,
    'Computer_graphics'
}

In [3]:
# создадим df со случайными значениями с заданными условиями
df = pd.DataFrame(np.random.randint(0, 100, size=(2000, 6)), columns=list(df_col))

In [4]:
# создадим два отдельных массива со временем и итоговой оценкой
df_time = pd.DataFrame(np.random.randint(15, 120, size=(2000,1)))
df_mark = pd.DataFrame(np.random.randint(0, 3, size=(2000,1)))
# переименуем эти массивы
df_time = df_time[0].rename('time') 
df_mark = df_mark[0].rename('mark')

In [5]:
# соединим исходные массивы данных
df = pd.concat(objs = [df, df_time, df_mark], axis = 1)
#df['mark'] = df['mark'].replace(0, 'ok') # Перевод целочисленных данных в категориальные
#df['mark'] = df['mark'].replace(1, 'good')
#df['mark'] = df['mark'].replace(2, 'perfect')

In [6]:
df

,db_engineering,Web_programming,MA,Math_modeling,Computer_graphics,Phys,time,mark
0,1,86,86,5,15,65,35,1
1,78,90,28,73,28,85,102,0
2,22,21,78,63,96,25,62,1
3,39,14,92,1,71,26,36,0
4,96,1,96,65,55,27,98,1
...,...,...,...,...,...,...,...,...
1995,50,32,54,48,11,59,24,2
1996,32,4,61,14,90,59,41,0
1997,9,92,1,8,75,41,45,0
1998,98,60,11,31,80,93,15,0


In [7]:
# разделим наш датасет на независимые и зависимые переменные
X = df[['db_engineering', 'Computer_graphics', 'Phys', 'Web_programming', 'MA', 'Math_modeling', 'time']]
Y = df[['mark']]

In [8]:
# разделим нашу выборку на тренировочную и тестовую (тестовой будет 20%)
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size = 0.2, random_state = 9526)

In [9]:
# определим наилучшие три фактора в исходных данных
selector = SelectKBest(chi2, k=3)
x_new = selector.fit_transform(x_train, y_train)
print(selector.get_feature_names_out()) # Вывод трех лучших признаков 

['Web_programming' 'MA' 'Math_modeling']


In [10]:
x_train

,db_engineering,Computer_graphics,Phys,Web_programming,MA,Math_modeling,time
1715,83,15,24,18,24,91,85
1720,71,49,58,72,18,58,31
512,62,39,3,92,17,89,105
1774,30,94,88,71,73,37,89
1838,90,22,42,13,13,53,39
...,...,...,...,...,...,...,...
800,30,50,42,28,51,87,32
1402,33,7,46,22,39,4,67
1503,64,57,37,37,69,16,76
311,96,43,61,42,29,96,108


In [11]:
# зададим гиперпараметры для нашей модели
iterations = 100
depth = 3
learning_rate = 1e-1
verbose = True

In [12]:
# создадим нашу модель классификации
model = CatBoostClassifier(iterations=iterations, depth=depth, learning_rate=learning_rate, verbose=verbose)

In [13]:
# обучим нашу модель на полученных данных
model.fit(x_train, y_train, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.0974063	total: 151ms	remaining: 14.9s
1:	learn: 1.0961814	total: 154ms	remaining: 7.53s
2:	learn: 1.0951920	total: 156ms	remaining: 5.05s
3:	learn: 1.0938551	total: 159ms	remaining: 3.81s
4:	learn: 1.0924072	total: 160ms	remaining: 3.05s
5:	learn: 1.0913811	total: 162ms	remaining: 2.54s
6:	learn: 1.0905928	total: 164ms	remaining: 2.18s
7:	learn: 1.0898830	total: 165ms	remaining: 1.9s
8:	learn: 1.0892400	total: 167ms	remaining: 1.69s
9:	learn: 1.0884622	total: 168ms	remaining: 1.52s
10:	learn: 1.0876841	total: 170ms	remaining: 1.37s
11:	learn: 1.0869791	total: 171ms	remaining: 1.25s
12:	learn: 1.0858244	total: 172ms	remaining: 1.15s
13:	learn: 1.0851112	total: 173ms	remaining: 1.06s
14:	learn: 1.0843460	total: 174ms	remaining: 988ms
15:	learn: 1.0836283	total: 176ms	remaining: 922ms
16:	learn: 1.0825653	total: 177ms	remaining: 863ms
17:	learn: 1.0819289	total: 178ms	remaining: 810ms
18:	learn: 1.0812378	total: 179ms	remaining: 762ms
19:	learn: 1.0803905	total: 180ms	remainin

In [14]:
preds_class = model.predict(x_test)
yt = np.array(y_test)
right = 0 # Проведем тестирование модели в ручном виде.
total = 0 # для этого прогоним тестовую выборку через модель
for i in range(400): # и сравним полученные значения
    if yt[[i]] == preds_class[[i]]:
        right += 1
    total += 1
print("Точность модели на тестовой выборке: ", right/total*100, "%")

Точность модели на тестовой выборке:  35.0 %


In [15]:
# сохраним модель в файл
model.save_model("class_1.cbm") 

In [16]:
# создадим функцию для прогноза оценки
def classify_mark():
    model1 = CatBoostClassifier()
    model1.load_model("class_1.cbm")
    print('Введите оценку по MA: ')
    MA = input()
    print('Введите оценку по Phys: ')
    Phys = input()
    print('Введите оценку по Math Modeling: ')
    MM = input()
    print('Введите оценку по Web Programming: ')
    WP = input()
    print('Введите оценку по DB Engineering: ')
    DBE = input()
    print('Введите оценку по Computer Graphics: ')
    CG = input()
    print('Введите время затраченное на работу: ')
    wTime = input()
    dataClassify = [MA, Phys, MM, WP, DBE, CG, wTime]
    res = model1.predict(dataClassify)
    if res == 0:
        print("удовлетворительно")
    elif res == 1:
        print("хорошо")
    else:
        print("отлично")

In [17]:
# протестируем функию проверки оценки
classify_mark()

Введите оценку по MA: 
100
Введите оценку по Phys: 
100
Введите оценку по Math Modeling: 
100
Введите оценку по Web Programming: 
100
Введите оценку по DB Engineering: 
100
Введите оценку по Computer Graphics: 
100
Введите время затраченное на работу: 
90
хорошо


#### Результат некорректный, так как выборка случайная.
#### Однако все работает, как и должно.